# 强化学习
在强化学习中，智能体agent在环境environment中做出观测observations做出动作actions,而后得的奖励rewards，行动的目的是获得最大期望长期受益。
## 策略搜寻（policy search）
策略:智能体决策行动的算法。策略可以是任何算法，甚至不是确定的算法。策略搜寻通常用遗传算法genetic algorithms。一次生成多个策略，然后按表现将策略排序，一一个特定的分位数舍去表现差的策略，让留下的策略组合产生新的子策略。（父辈的拷贝加入一些随机变换），留下的策略与子策略构成第二代策略集。然后迭代进行，直至找到最优策略。<br/>
另一种常用的策略搜寻算法称为策略梯度算法。首先奖励对策略参数的梯度，然后根据梯度上升的方法调整参数.
## OpenAI Gym简介
在真实场景中训练强化学习是非常昂贵且费时的，因此训练通常是在模拟环境中进行的。OpenAI Gym是一个复杂而包罗万象的模拟环境。

In [2]:
import gym
env=gym.make('CartPole-v0')
obs=env.reset()
env.render()
img=env.render(mode='rgb_array')
img.shape
env.action_space

action=1
obs,reward,done,info=env.step(action)



def basic_policy(obs):
    angle=obs[2]
    return 0 if angle<0 else 1
total=[]
for episode in range(500):
    obs=env.reset()
    for step in range(1000):
        action=basic_policy(obs)
        obs,reward,done,info=env.step(action)
        episode_rewards+=reward
        if done:
            break
    totals.eppend(episode_rewards)
    
import numpy as np
np.mean(totals),np.std(totals),np.min(totals),np.max(totals)



AttributeError: module 'gym' has no attribute 'make'

## 神经网络方法
神经网络以观测为输入，动作为输出，更准确地说，是估计每个动作的概率，然后根据概率随机选择动作。在CartPole中，由于只有两个动作，所以只需要一个输出神经元，输出为action 0（左）该率P.根据概率随机选择的动作，而不是直接根据概率选择的动作的原因在于，这个方法可以使智能体在发掘新动作和利用现有动作之间取得平衡。需要注意的是，在这个特殊的环境中，过去的动作和观测，可以被忽略，因为每个动作都包含这个环境的完全状态。如果环境中存在隐藏动作，那么过去的动作和观测也需要被考虑进去。如：CartPole的位置。

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
n_inputs=4
n_hidden=4
n_outputs=1
initializer=tf.contrib.layers.variance_scaling_initializer()

X=tf.placeholder(tf.float32,shape=[None,n_inputs])
hidden=fully_connected(X,n_hidden,activation_fn=tf.nn.elu,weights_initializer=initializer)
logits=fully_connected(hidden,n_outputs,activation_fn=None,weights_initializer=initializer)
outputs=tf.nn.sigmoid(logits)

p_left_and_right=tf.concat(value=[outputs,1-outputs],axis=1)
action=tf.multinomial(tf.log(p_left_and_right),num_samples=1)#按概率随机选择动作，多项式随机
init=tf.global_variables_initializer()


## 估计动作：信度指派问题

如果知道每一步的最优动作，我们当然可以像训练监督学习那样训练神经网络。但问题在于，在强化学习中，智能体只能通过奖励活动反馈，而奖励是稀疏而且有延迟的。例如，如果要在100步内平衡杆子，它该如何知道100步中哪些是好的，哪些是坏的，而如果最终杆子在第100步时坠落，那么问题并不是完全出自第100步，这种问题即被称为信度指派问题。当智能体获得奖励，它并不能确定哪一步是可信的，哪一步是可疑的。<br/>
解决该问题最常用的一种方法是：根据之后获得的奖励总和来估计该动作，通常会以一个折旧率来刻画时间价值。如一个连续三次向右的动作，第一次奖励为10，第二次奖励为0，最终奖励为-50，折旧率为0.8，则第一个动作的总得分为：$10+r\times 0+r^2\times(-50)=-22$,通常折旧率为0.95或0.99，折旧率为0.95时，10步后的奖励相当于即时奖励的一半，折旧率为0.99，相当于69步后的奖励相当于即时奖励的一半。在CartPole环境中，动作具有短期效应，选择0.95的折旧率是合理的。当然好的动作如果跟随的是一系列坏的动作，那么好的动作的得分也会很低，可是如果时间足够长的话，平均来说，好的动作的平均得分高于坏的动作。因此为得到相对可靠的动作得分，我们必须多次运行并标准化所有得分。最后，得分为负的为坏动作，为正的为好动作。
## 策略梯度
策略梯度算法指的是跟随更高得分的梯度从而优化策略的参数方法。最流行的策略梯度算法是REINFORCE算法，以下是它的一个变体：
1. 首先，然神经网络策略测试游戏数次，并在每一步计算使选中的动作更可能的梯度，但并不应用这些梯度。
1. 运行游戏数次，计算每个动作的得分。
3. 如果某个动作的得分为正，则希望应用之前计算的梯度使得该动作更容易被选中，反正，则应用相反的梯度。方法就是将梯度向量与对应动作得分相乘。
4. 最后，计算所有梯度向量的均值，并用其执行梯度下降方法。


In [ ]:
y=1.-tf.to_float(action)#目标概率
learning_rate=0.01

cross_entropy=tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits)
optimizer=AdamOptimizer(learning_rate=learning_rate)
grad_and_vars=optimizer.compute_gradients(cross_entropy)#得到梯度和变量

gradients=[grad for grad,variable in gard_and_vars]#取出梯度
#在执行阶段，算法会运行所选择的策略，而在运行的每一步，算法也会评估这些梯度向量并且存储
#向量的值。几次运行之后，算法会调整这些梯度（与动作得分相乘并标准化），并计算调整梯度的
#均值。然后，将调整的梯度代入之前的优化器，最后执行优化过程，这意味着我们需要为每个梯度
#向量设一个占位符。此外，我们还必须创建更新梯度的操作，为此我们要调用优化器的apply_gradients
#函数，创建一个包含更新梯度的列表。
gradient_placeholders=[]
grads_and_vars_feed=[]
for grad,variable in grads_and_vars:
    gradient_placeholder=tf.placeholder(tf.float32,shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grad_and_vars_feed.append((gradient_placeholder),variable)

training_op=optimizer.apply_gradients(grads_and_vars_feed)




In [ ]:
import tensorflow as tf
from tf.contrib.layers import fully_connected
n_inputs=4
n_hidden=4
n_outputs=1
initializer=tf.contrib.layers.variance_scaling_initializer()
learning_rate=0.01

X=tf.placeholder(tf.float32,shape=[None,n_inputs])
hidden=fully_connected(X,n_hidden,activation_fn=tf.nn.elu,weights_initializer=initializer)
logits=fully_connected(hidden,n_outputs,activaion_fn=None,weights_initializer=initializer)
outputs=tf.nn.sigmoid(logits)

p_left_and_right=tf.concat(axis=1,values=[outputs,1-outputs])
action=tf.multinomial(tf.log(p_left_and_right),num_samples=1)
y=1.-tf.to_float(action)
cross_entropy=tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits)
optimizer=tf.train.AdamOptimizer(learning_rate)
grads_and_vars=optimizer.compute_gradients(cross_entropy)
gradients=[grad for gard,variable in grads_and_vars]

gradient_placeholders=[]
grads_and_vars_feed=[]
for grad,variable in grad_and_vars:
    gradient_placeholder=tf.placeholder(tf.float32,shape=grad.get_shape())
    gradient_placholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder,variable))
training_op=optimizer.apply_gradents(grads_and_vars_feed)

init=tf.global_variables_initializer()
saver=tf.train.Saver()


#执行阶段
def discount_rewards(rewards,discount_rate):
    discounted_rewards=np.empty(len(rewards))
    cumulative_rewards=0
    for step in reversed(range(len(rewards))):
        cumulative_rewards=rewards[step]+cumulative_rewards*discount_rate
        discounted_rewards[step]=cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards,discount_rate):
    all_discounted_rewards=[discount_rewards(rewards) for rewards in all_rewards]
    flat_rewards=np.concatenate(all_discounted_rewards)
    reward_mean=flat_rewards.mean()
    reward_std=flat_rewards.std()
    return [(discounted_rewards-reward_mean)/reward_std
           for discounted_rewards in all_discounted_rewards]


n_iterations=250
n_max_steps=1000
n_games_per_update=10
save_iterations=10
discount_rate=0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards=[]
        all_grdients=[]
        for game in range(n_game_per_update):
            current_rewards=[]
            current_gradients=[]
            obs=env.reset()
            for step in range(n_max_steps):
                action_val,gradients_val=sess.run(
                [action,gradients],feed_dict={X:obs.reshape(1,n_inputs)})#one obs
                obs,reward,done,info=env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current__rewards)
            all_gradients.append(current_graidents)
            #运行10episodes,并更新策略
            all_rewards=discount_and_nomalize_rewards(all_rewards)
            feed_dict={}
            for val_index,grad_placeholder in enumerate(gradient_placholders):
                mean_gradients=np.mean(
                [reward*all_gradients[game_index][step][var_index] for game_index,rewards 
                in enumerate(all_rewards) for step,reward in enumerate(rewards)],axis=0)
                feed_dict[grad_placeholder]=mean_gradients
            sess.run(training_op,feed_dict=feed_dict)
            if iteration%save_iterations==0:
                saver.save(sess,'./my_policy_net_pg.ckpt')
            